In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read trace data

In [ ]:
root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")

brains = ["brain1", "brain2"]
measures = ["curvature", "torsion"]
max_id = 300

d = []

for brain in brains:
    data_dir = os.path.join(experiment_dir, "data", brain)
    segments_swc_dir = os.path.join(data_dir, "segments_swc")
    trace_data_dir = os.path.join(data_dir, "trace_data", "14")
    print(f"Directory where swcs reside: {segments_swc_dir}")
    
    for i in np.arange(0, max_id):
        i = int(i)
        trace_data_path = os.path.join(trace_data_dir, f"{i}.npy")
        if os.path.exists(trace_data_path) is True:
            trace_data = np.load(trace_data_path, allow_pickle=True)
            print(f"Loaded segment {i}")
            
            for node in trace_data:
                seg_length = node["seg_length"]
                log_seg_length = np.log10(seg_length)
                for measure in measures:
                    _measure = node[f"mean_{measure}"]
                    magnitude = "non-zero" if _measure > 1e-16 else "zero"
                    d.append({"brain": brain, "log_seg_length": log_seg_length, "measure": measure, "magnitude": magnitude})
df = pd.DataFrame(d)

# Plot zero/non-zero magnitude densities as a function of `log` segment length

In [ ]:
sns.set_theme()
sns.set_context("paper")
sns.set(font_scale=1.2)

g = sns.FacetGrid(df, col="measure", row="brain", hue="magnitude", height=4)
g.map(sns.kdeplot, "log_seg_length")
g.set_axis_labels("log segment length", "density")
g.add_legend(title="Magnitude")

axes = g.axes

for i, brain in enumerate(brains):
    brain_df = df[df["brain"] == brain]
    for j, measure in enumerate(measures):        
        measure_df = brain_df[brain_df["measure"] == measure]
        zero_measure = measure_df[measure_df["magnitude"] == "zero"]
        alpha_measure = len(zero_measure)/len(measure_df)
        print(f"{brain}, {measure}, alpha = {alpha_measure}")
        
        ax = axes[i, j]
        ax.set_title(r"Brain %d %s ($\alpha = %.2f$)" % (i+1, measure, alpha_measure))

g.savefig(os.path.join(experiment_dir, "figures", "histograms.eps"))
g.savefig(os.path.join(experiment_dir, "figures", "histograms.jpg"))